In [1]:
import tensorflow as tf


In [46]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
import os
import glob
import cv2
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
import dlib

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.utils import to_categorical


In [11]:
x = np.load("test_data_gray.npy")
y = np.load("test_data_label_gray.npy")
x.shape, y.shape

((20979, 224, 224), (20979,))

## creating training/validation data set from extracted images

In [2]:
def create_training_data():
    training_data = []
    training_data_label = []
    IMG_SIZE = 224
    path = "D:/training_data"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    for person in tqdm(os.listdir(path)):
        for state in os.listdir(path + "/" + person):
            count = 0
            filter = 5
            for img in os.listdir(path + "/" + person + "/" + state):
                if count % filter == 0:
                        
                    #check if face is recognizable
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                        # Create landmark object
                        landmarks = predictor(image=img, box=face)
                        if landmarks.part(42).x != '':
                            if state == "0":
                                training_data_label.append(np.array(0))
                            else:
                                training_data_label.append(np.array(1))
                            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                            training_data.append(np.array(img))
                        break
                    
                count += 1
                        
                
    np.save("training_data_shrunk", training_data)
    np.save("training_label_shrunk", training_data_label)
    
    print("Done!")
    
#create_training_data()

In [3]:
def create_test_data():
    test_data = []
    test_data_label = []
    IMG_SIZE = 224
    path = "D:/test_data"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    for person in tqdm(os.listdir(path)):
        for state in os.listdir(path + "/" + person):
            count = 0
            filter = 5
            for img in os.listdir(path + "/" + person + "/" + state):
                if count % filter == 0:
                    #check if face is recognizable
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                     # Create landmark object
                        landmarks = predictor(image=img, box=face)
                        if landmarks.part(42).x != '':
                            if state == "0":
                                test_data_label.append(np.array(0))
                            else:
                                test_data_label.append(np.array(1))
                            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                            test_data.append(np.array(img))
                        break
                count += 1
                
    np.save("test_data_shrunk", test_data)
    np.save("test_label_shrunk", test_data_label)
    
    print("Done!")
    
#create_test_data()

In [6]:
create_training_data()
create_test_data()

100%|██████████| 48/48 [08:45<00:00, 10.94s/it]


Done!


100%|██████████| 12/12 [01:52<00:00,  9.36s/it]


Done!


# Data and Label

In [10]:
x_train = np.load("training_data_RBG_v3.npy")

y_train = np.load("training_data_label_2_classes.npy")

x_train.shape, y_train.shape

((84973, 224, 224, 3), (84973,))

In [11]:
x_test = np.load("test_data_RBG_v3.npy")

y_test = np.load("test_data_label_2_classes.npy")

x_test.shape, y_test.shape

((21200, 224, 224, 3), (21200,))

In [4]:
xtrain_MLP = np.load("training_data_RATIO_v4.npy")

xtest_MLP = np.load("test_data_RATIO_v4.npy")

xtest_MLP.shape, xtrain_MLP.shape

((21200, 6), (84973, 6))

In [6]:
y_train = to_categorical(y_train, num_classes=3)

y_train[0]

array([1., 0., 0.], dtype=float32)

In [7]:
y_test = to_categorical(y_test, num_classes=3)
print(y_test[0])
y_test.shape

[1. 0. 0.]


(38386, 3)

In [5]:
x_train, y_train, xtrain_MLP = shuffle(x_train, y_train, xtrain_MLP)
x_train.shape, y_train.shape, xtrain_MLP.shape

((84973, 224, 224, 3), (84973,), (84973, 6))

# Model implementation

In [12]:
def CNN_MLP():
    inputs = tf.keras.Input(shape=(6,))
    layer = layers.Dense(6, activation='relu')(inputs)
    resnet = tf.keras.applications.ResNet50(weights="imagenet", include_top=False,
                                            input_tensor=tf.keras.Input(shape=(224, 224, 3)))
    flatten = layers.Flatten()(resnet.output)
    combined = layers.concatenate([flatten, layer])
    output = layers.Dense(1, activation='sigmoid')(combined)
    
    model = tf.keras.Model(inputs=[resnet.input, inputs], outputs=output)
    return model

In [13]:
model = CNN_MLP()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)


model.fit([x_train, xtrain_MLP], 
          y_train,
        epochs=20,
         batch_size=32,
         validation_data=([x_test,xtest_MLP], y_test),
         callbacks=[EarlyStopping(monitor='val_loss', 
                                             min_delta=0.0001, 
                                             patience=3, 
                                             restore_best_weights=True)])

Train on 84973 samples, validate on 21200 samples
Epoch 1/20
84973/84973 [==============================] - 797s 9ms/sample - loss: 0.6685 - accuracy: 0.9061 - AUC: 0.9519 - val_loss: 6.1728 - val_accuracy: 0.5097 - val_AUC: 0.4723
Epoch 2/20
84973/84973 [==============================] - 809s 10ms/sample - loss: 0.4577 - accuracy: 0.9527 - AUC: 0.9793 - val_loss: 6.3957 - val_accuracy: 0.6348 - val_AUC: 0.4720
Epoch 3/20
84973/84973 [==============================] - 831s 10ms/sample - loss: 0.1610 - accuracy: 0.9777 - AUC: 0.9922 - val_loss: 2.6506 - val_accuracy: 0.5908 - val_AUC: 0.4363
Epoch 4/20
84973/84973 [==============================] - 813s 10ms/sample - loss: 0.1068 - accuracy: 0.9837 - AUC: 0.9941 - val_loss: 3.4825 - val_accuracy: 0.5758 - val_AUC: 0.4209
Epoch 5/20
84973/84973 [==============================] - 863s 10ms/sample - loss: 0.1054 - accuracy: 0.9849 - AUC: 0.9941 - val_loss: 5.9683 - val_accuracy: 0.6054 - val_AUC: 0.4523
Epoch 6/20
84973/84973 [============

In [16]:
model.save("shrunk_10N")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: shrunk_10N\assets


In [9]:
def ResNet50():
    resnet = tf.keras.applications.ResNet50(weights="imagenet", include_top=False,
                                            input_tensor=tf.keras.Input(shape=(224, 224, 3)))
    flatten = layers.Flatten()(resnet.output)
    dense = layers.Dense(512, activation="relu")(flatten)
    dropout = layers.Dropout(0.5)(dense)
    output = layers.Dense(3, activation='softmax')(dropout)
    model = tf.keras.Model(inputs=resnet.input, outputs=output)
    return model

In [48]:
def CNN():
    inputs = tf.keras.Input(shape=(224, 224, 1))
    layer = layers.Conv2D(8, 3, padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.ReLU()(layer)    
    layer = layers.Conv2D(16, 3, padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.ReLU()(layer)
    layer = layers.MaxPooling2D()(layer)
    
    layer = layers.Conv2D(32, 3, padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.ReLU()(layer)
    layer = layers.Conv2D(64, 3, padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.ReLU()(layer)
    layer = layers.MaxPooling2D()(layer)
    
    
    layer = layers.Flatten()(layer)
    layer = layers.Dense(128, activation='relu')(layer)
    #layer = layers.Dropout(0.3)(layer)
    layer = layers.Dense(1, activation = 'sigmoid')(layer)
    
    model = tf.keras.Model(inputs=inputs,outputs=layer)
    
    return model

In [57]:
def MLP():
    inputs = tf.keras.Input(shape=(6,))
    layer = layers.Dense(32, activation='sigmoid')(inputs)
    layer = layers.Dense(64, activation='sigmoid')(layer)
    layer = layers.Dense(128, activation='sigmoid')(layer)
    layer = layers.Dense(256, activation='sigmoid')(layer)
    layer = layers.Dense(512, activation='sigmoid')(layer)
    layer = layers.Dense(256, activation='sigmoid')(layer)
    layer = layers.Dense(64, activation='sigmoid')(layer)
    layer = layers.Dense(32, activation='sigmoid')(layer)
    layer = layers.Dense(1, activation='sigmoid')(layer)
    
    model = tf.keras.Model(inputs=inputs, outputs=layer)
    return model

In [51]:
train_target = np.load("training_data_label_v3.npy")
test_target = np.load("test_data_label_v3.npy")
train_target.shape, test_target.shape

((84973,), (21200,))

In [58]:
mlp = MLP()

mlp.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

mlp.fit(xtrain_MLP,
        train_target,
        epochs=20,
         batch_size=32,
         validation_data=(xtest_MLP, test_target),
         callbacks=[EarlyStopping(monitor='val_loss', 
                                             min_delta=0.0001, 
                                             patience=5, 
                                             restore_best_weights=True)])


Train on 84973 samples, validate on 21200 samples
Epoch 1/20
84973/84973 [==============================] - 10s 120us/sample - loss: 0.0781 - accuracy: 0.3262 - AUC: 0.5001 - val_loss: -0.1744 - val_accuracy: 0.3644 - val_AUC: 0.5000
Epoch 2/20
84973/84973 [==============================] - 9s 109us/sample - loss: 0.0729 - accuracy: 0.3262 - AUC: 0.4996 - val_loss: -0.1892 - val_accuracy: 0.3644 - val_AUC: 0.5000
Epoch 3/20
84973/84973 [==============================] - 9s 109us/sample - loss: 0.0729 - accuracy: 0.3262 - AUC: 0.4984 - val_loss: -0.1864 - val_accuracy: 0.3644 - val_AUC: 0.5000
Epoch 4/20
84973/84973 [==============================] - 10s 113us/sample - loss: 0.0735 - accuracy: 0.3262 - AUC: 0.4992 - val_loss: -0.1747 - val_accuracy: 0.3644 - val_AUC: 0.5000
Epoch 5/20
84973/84973 [==============================] - 10s 114us/sample - loss: 0.0728 - accuracy: 0.3262 - AUC: 0.4998 - val_loss: -0.1513 - val_accuracy: 0.3644 - val_AUC: 0.5000
Epoch 6/20
84973/84973 [========

In [14]:
model = ResNet50()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'AUC']
)


model.fit(x_train[:20000],
        y_train[:20000],
        epochs=20,
         batch_size=32,
         validation_data=(x_test[:8000], y_test[:8000]),
         callbacks=[EarlyStopping(monitor='val_loss', 
                                             min_delta=0.0001, 
                                             patience=3, 
                                             restore_best_weights=True)])

Train on 20000 samples, validate on 8000 samples
Epoch 1/20
20000/20000 [==============================] - 200s 10ms/sample - loss: 1.3492 - accuracy: 0.6877 - AUC: 0.8506 - val_loss: 1.3312 - val_accuracy: 0.5159 - val_AUC: 0.7048
Epoch 2/20
20000/20000 [==============================] - 195s 10ms/sample - loss: 0.2288 - accuracy: 0.9218 - AUC: 0.9848 - val_loss: 0.5232 - val_accuracy: 0.8364 - val_AUC: 0.9497
Epoch 3/20
20000/20000 [==============================] - 195s 10ms/sample - loss: 0.1105 - accuracy: 0.9636 - AUC: 0.9956 - val_loss: 0.5387 - val_accuracy: 0.8708 - val_AUC: 0.9575
Epoch 4/20
20000/20000 [==============================] - 194s 10ms/sample - loss: 0.1332 - accuracy: 0.9566 - AUC: 0.9938 - val_loss: 7.3102 - val_accuracy: 0.6053 - val_AUC: 0.7214
Epoch 5/20
20000/20000 [==============================] - 194s 10ms/sample - loss: 0.1113 - accuracy: 0.9653 - AUC: 0.9954 - val_loss: 0.2997 - val_accuracy: 0.9396 - val_AUC: 0.9809
Epoch 6/20
20000/20000 [============

In [15]:
model.save("CNN_fake")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: CNN_fake\assets


In [19]:
model.evaluate(x_test, y_test)

16992/21200 [=======================>......] - ETA: 10s - loss: 6298269.3746 - accuracy: 0.6981 - AUC: 0.5000

KeyboardInterrupt: 

In [ ]:
x=[]
y=[]
r=[]

for i in range(len(x_train)):
    if i % 5 == 0:
        x.append(x_train[i])
        y.append(y_train[i])
        r.append(xtrain_MLP)
        
x = np.array(x)


In [13]:
model2 = CNN_MLP()

model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
)

model2.fit([x_train, xtrain_MLP], y_train,
            shuffle =True,
            epochs=20,
            batch_size = 64,
            validation_data=([x_test,xtest_MLP], y_test),
            callbacks=[EarlyStopping(monitor='val_loss', 
                                                 min_delta=0.0001,
                                                 patience=4,
                                                 restore_best_weights=True)])



Train on 84973 samples, validate on 21200 samples
Epoch 1/20
   64/84973 [..............................] - ETA: 6:00:12WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: 


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[64,1024,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_3/conv4_block2_3_bn/FusedBatchNormV3 (defined at <ipython-input-13-bc600e10ffea>:17) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[metrics/AUC/assert_greater_equal/Assert/AssertGuard/pivot_f/_3/_19]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[64,1024,14,14] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_3/conv4_block2_3_bn/FusedBatchNormV3 (defined at <ipython-input-13-bc600e10ffea>:17) ]]
Hint: If you want to see a list of

In [8]:
model2.fit([x_train[20001:45000], xtrain_MLP[20001:45000]], y_train[20001:45000],
            shuffle =True,
            epochs=1,
            batch_size = 32,
            validation_data=([x_test,xtest_MLP], y_test)
            #callbacks=[EarlyStopping(monitor='val_loss', 
             #                                    min_delta=0.0001,
              #                                   patience=4,
             #                                    restore_best_weights=True)])
          )

model2.fit([x_train[45001:], xtrain_MLP[45001:]], y_train[45001:],
            shuffle =True,
            epochs=1,
            batch_size = 32,
            validation_data=([x_test,xtest_MLP], y_test)
            #callbacks=[EarlyStopping(monitor='val_loss', 
             #                                    min_delta=0.0001,
              #                                   patience=4,
             #                                    restore_best_weights=True)])
          )

Train on 24999 samples, validate on 21200 samples
24999/24999 [==============================] - 273s 11ms/sample - loss: 0.2663 - accuracy: 0.8988 - val_loss: 2.5384 - val_accuracy: 0.4941
Train on 39972 samples, validate on 21200 samples
39972/39972 [==============================] - 412s 10ms/sample - loss: 0.1050 - accuracy: 0.9655 - val_loss: 2.7012 - val_accuracy: 0.6634


In [9]:
model2.evaluate([x_test,xtest_MLP], y_test)

21200/21200 [==============================] - 50s 2ms/sample - loss: 2.7012 - accuracy: 0.6634


[2.701225096295141, 0.6634434]

In [10]:
model2.save("Resnet50_MLP_dropout")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Resnet50_MLP_dropout\assets


In [88]:
model2.evaluate([x_test,xtest_MLP], y_test)

21200/21200 [==============================] - 52s 2ms/sample - loss: 0.9181 - accuracy: 0.4990


[0.9181041660398808, 0.49896225]

# Past Results

In [26]:
model = ResNet50()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [27]:
model.fit(x_train,
        y_train,
        epochs=20,
         batch_size=32,
         validation_data=(x_test, y_test),
         callbacks=[EarlyStopping(monitor='val_loss', 
                                             min_delta=0.0001, 
                                             patience=5, 
                                             restore_best_weights=True)])

Train on 84974 samples, validate on 21202 samples
Epoch 1/20
84974/84974 [==============================] - 776s 9ms/sample - loss: 0.3732 - accuracy: 0.8983 - val_loss: 4.2246 - val_accuracy: 0.3185
Epoch 2/20
84974/84974 [==============================] - 770s 9ms/sample - loss: 0.1239 - accuracy: 0.9687 - val_loss: 5.3317 - val_accuracy: 0.3764
Epoch 3/20
84974/84974 [==============================] - 768s 9ms/sample - loss: 0.0667 - accuracy: 0.9831 - val_loss: 6.7455 - val_accuracy: 0.2343
Epoch 4/20
84974/84974 [==============================] - 765s 9ms/sample - loss: 0.0418 - accuracy: 0.9880 - val_loss: 6.0145 - val_accuracy: 0.2432
Epoch 5/20
84974/84974 [==============================] - 771s 9ms/sample - loss: 0.0335 - accuracy: 0.9897 - val_loss: 5.7112 - val_accuracy: 0.2993
Epoch 6/20
84974/84974 [==============================] - 765s 9ms/sample - loss: 0.0236 - accuracy: 0.9927 - val_loss: 14.7564 - val_accuracy: 0.3088


In [28]:
model.save("Resnet50")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: Resnet50\assets


In [39]:
model2 = CNN_MLP()

model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [40]:
model2.fit([x_train,xtrain_MLP], y_train,
        epochs=20,
         batch_size = 32,
           validation_data=([x_test,xtest_MLP], y_test),
             callbacks=[EarlyStopping(monitor='val_loss', 
                                             min_delta=0.0001,
                                             patience=4,
                                             restore_best_weights=True)])

Train on 84973 samples, validate on 21200 samples
Epoch 1/20
Train on 84973 samples, validate on 21200 samples
Epoch 1/20
84973/84973 [==============================] - 1053s 12ms/sample - loss: 1.0580 - accuracy: 0.8424 - val_loss: 4.6135 - val_accuracy: 0.3183
Epoch 2/20

84973/84973 [==============================] - 893s 11ms/sample - loss: 0.6184 - accuracy: 0.9425 - val_loss: 6.2376 - val_accuracy: 0.3168
Epoch 3/20Epoch 3/20

84973/84973 [==============================] - 871s 10ms/sample - loss: 0.4333 - accuracy: 0.9611 - val_loss: 4.2994 - val_accuracy: 0.3287
Epoch 4/20
Epoch 4/20
84973/84973 [==============================] - 864s 10ms/sample - loss: 0.6344 - accuracy: 0.9614 - val_loss: 14.6655 - val_accuracy: 0.2765
Epoch 5/20
Epoch 5/20
84973/84973 [==============================] - 833s 10ms/sample - loss: 0.5751 - accuracy: 0.9601 - val_loss: 9.4857 - val_accuracy: 0.2944
Epoch 6/20
84973/84973 [==============================] - 833s 10ms/sample - loss: 0.5751 - accura

In [41]:
model2.save("Resnet50_MLP")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: Resnet50_MLP\assets
INFO:tensorflow:Assets written to: Resnet50_MLP\assets


INFO:tensorflow:Assets written to: Resnet50_MLP\assets
INFO:tensorflow:Assets written to: Resnet50_MLP\assets
